In [ ]:
# !pip install peft

In [ ]:
# !pip install matplotlib

In [ ]:
# https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms
# https://anirbansen2709.medium.com/finetuning-llms-using-lora-77fb02cbbc48
# https://github.com/microsoft/LoRA/tree/main/examples/NLG
# https://github.com/huggingface/peft


In [ ]:
# !pip install wandb

In [ ]:
# !pip install deepspeed

In [ ]:
# !DS_BUILD_OPS=1 pip install --no-cache-dir deepspeed

In [ ]:
# deepspeed.__version__

In [ ]:
# !pip show deepspeed

In [ ]:
# !pip install accelerate

In [45]:
# !pip install --upgrade huggingface_hub

In [1]:
import sys
sys.path.append('/home/blatova/lca-solvers/')

In [2]:
# import finetuning_pipeline

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: ekaterina-blatova (org12345). Use `wandb login --relogin` to force relogin


True

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import sys
import gc
import math
import re
import string
import numpy as np
import random
from collections import Counter
import os
import deepspeed
import pickle

import torch
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, LambdaLR
from accelerate import Accelerator, DeepSpeedPlugin

from transformers import AutoTokenizer, AutoModelForCausalLM
#CHANGED_LAST
from transformers import get_cosine_schedule_with_warmup
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

import sys
sys.path.append('/home/blatova/lca-solvers/')

from data_filters.repo_snapshot_filter_stack import SnapshotFilterStack
from data_classes.datapoint_composed import DatapointComposed
from data_loading.raw_train_dataset_loading import DataLoaderTrainRaw
from data_loading.composer_train_dataset_loading import DataLoaderTrainComposed
from context_composers.context_composer_path_distance import ContextComposerPathDistance
from data_classes.datapoint_composed import DatapointComposed
from datasets import load_dataset
from huggingface_hub import HfApi, HfFolder, Repository, RepoCard

punctuation = string.punctuation
punctuation += "–—‘’“”…"
pattern = re.compile(r'^[a-zA-Z0-9\s' + re.escape(punctuation) + r']*$')

[2024-07-11 20:06:04,853] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


In [6]:
# CUDA_VISIBLE_DEVICES=6,7 jupyter notebook

In [7]:
!nvidia-smi

Thu Jul 11 20:06:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0 Off |                  Off |
|  0%   30C    P8              14W / 450W |      6MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
DEVICES_TO_TRAIN=[3]
WORLD_SIZE= len(DEVICES_TO_TRAIN)

# os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(x) for x in DEVICES_TO_TRAIN)


In [9]:
# os.environ['CUDA_VISIBLE_DEVICES']

In [10]:
# !kill -9 1694536

In [11]:
torch.cuda.is_available()

True

In [12]:
torch.cuda.device_count()

1

In [13]:
# !pip install flash-attn --no-build-isolation

In [14]:
# loader = DataLoader(cache_dir="/home/blatova/cache_dir")
# loader = DataLoaderTrainComposed(
#         hf_path='JetBrains-Research/context-py-train',
#         hf_config=None,
#         cache_dir='/mnt/data2/shared-data/lca/hf_cache2/',
#     )
composer = ContextComposerPathDistance()
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", 
                                             trust_remote_code=True, torch_dtype=torch.bfloat16, device_map = "auto", attn_implementation="flash_attention_2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Lora parameters

https://medium.com/@manyi.yim/more-about-loraconfig-from-peft-581cf54643db

https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms

https://abvijaykumar.medium.com/fine-tuning-llm-parameter-efficient-fine-tuning-peft-lora-qlora-part-2-d8e23877ac6f

AdaMix Lora: https://arxiv.org/pdf/2210.17451v1

Lora-based mixture of experts: https://arxiv.org/pdf/2404.15159

I believe the modules you've mentioned: gate_proj, down_proj, up_proj, q_proj, v_proj, k_proj, and o_proj, are all linear layers. So, they should be included in the list for LoRA finetuning to get the best results, that compare to full finetuning.

This is based on the QLoRA paper: "We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers is required to match full finetuning performance."

To check which layers are linear, you can load the model with the transformers library in Python and then just print(model). The layers to look for will be labeled Linear or Linear4bit if you load the model in 4-bit. While I can't say this is the only way, it's how I've always done it.

For QLoRA finetuning, I've used my own custom script, so I wrote a function that returns a list to simplify the "finding layers to finetune" process. However, keep in mind, this function will work properly only if the model is loaded in 4-bit using the transformers library:

```def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()
    
    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]
            
            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)```
From: https://www.reddit.com/r/LocalLLaMA/comments/15sgg4m/what_modules_should_i_target_when_training_using/

```import re
model_modules = str(model.modules)
pattern = r'\((\w+)\): Linear'
linear_layer_names = re.findall(pattern, model_modules)

names = []
# Print the names of the Linear layers
for name in linear_layer_names:
    names.append(name)
target_modules = list(set(names))```


Better result about layers and parameters fine-tuning:

https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms : All linear layers

***Other parameters:***

***Optimizer: Commonly used AdamW***

r=16 => alpha = 16 (That’s why the authors set α to the first r and do not tune it. The default of α is 8.)

***r=32 => alpha = 32***

Dropout rate: 

https://ar5iv.labs.arxiv.org/html/2403.00812

https://arxiv.org/pdf/2403.00812

https://arxiv.org/pdf/2404.09610

Hence, we only introduce the dropping in the latter half of layers in decoder-only models and the apparent performance improvement emerges again.

***Dropout rate: Start without dropout? Usually used 0.1 ***

bias: ???




In [15]:
#CHANGED
# ds = load_dataset('JetBrains-Research/context-py-train', 'path_distance_relevant', cache_dir = '/mnt/data2/shared-data/lca/hf_cache2/')
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=32, lora_alpha=32, lora_dropout=0.1, target_modules='all-linear'
)
# peft_config = LoraConfig(
#             lora_alpha=args.lora_alpha,
#             lora_dropout=args.lora_dropout,
#             r=args.lora_r,
#             bias="none",
#             task_type=TaskType.CAUSAL_LM,
#             target_modules="all-linear"
#         )
# model_lora = get_peft_model(model, peft_config)
# model_lora_no_parallel = torch.nn.DistributedDataParallel(model_lora, DEVICES_TO_TRAIN)
# model_lora = deepspeed.init_inference(model_lora_no_parallel, mp_size=len(DEVICES_TO_TRAIN)) 
# model.print_trainable_parameters()
criterion = CrossEntropyLoss()

In [16]:
SUFFIX='lr4e-5_old_scheduler_with_warmup_3_epochs_relevant'
def save_checkpoint(checkpoint_num, model, val_loss_whole_input , val_loss_completion, validation_dataset, suffix=SUFFIX):
    #CHANGED
    repo_name = f"model_{suffix}_v{checkpoint_num}"
    model.push_to_hub(repo_name)
    model_description = f"""
    
    ## Evaluation results
    
    Validation loss on the whole input: {val_loss_whole_input}

    Validation loss on completion: {val_loss_completion}
    
    """
    model_card = RepoCard(model_description)
    model_card.push_to_hub(f"ekaterina-blatova-jb/{repo_name}")
    # if checkpoint_num ==0:
    #     validation_dataset.push_to_hub(f"val_dataset_{suffix}")
        
    


In [17]:
def get_standard_val_dataset(val_indices_list):
    return dataset['train'].select(val_indices_list)

def write_val_dataset(std_val_dataset_version, val_dataset, validation_repos, val_indices_list):
    torch.save(val_dataset, f'std_val_dataset/std_val_dataset_{std_val_dataset_version}.pt')    
    with open(f"std_val_dataset/std_val_dataset_{std_val_dataset_version}_repos.pkl", "wb") as repos: 
        pickle.dump(validation_repos, repos)
    with open(f"std_val_dataset/std_val_dataset_{std_val_dataset_version}_indices.pkl", "wb") as indices: 
        pickle.dump(val_indices_list, indices)
    val_dataset.push_to_hub(f"val_dataset_std_{std_val_dataset_version}")
    

def read_val_dataset(std_val_dataset_version):
    with open(f"std_val_dataset/std_val_dataset_{std_val_dataset_version}_repos.pkl", "rb") as repos: 
        validation_repos = pickle.load(repos)
    with open(f"std_val_dataset/std_val_dataset_{std_val_dataset_version}_indices.pkl", "rb") as indices: 
        val_indices_list = pickle.load(indices)
    val_dataset = get_standard_val_dataset(val_indices_list)
    return val_dataset, validation_repos, val_indices_list

In [18]:
# model_lora.hf_device_map

In [19]:
#6000 tokens => OOM
ACCUM_STEPS_NUM = 64
# VALIDATION_STEPS = 16
VALIDATION_PERIOD = 10
# CONTEXT_MAX_LEN_CHARS = 5000
CONTEXT_MAX_LEN_TOKENS = 2000
EMA_PERIOD = 40
EMA_ALPHA = 2/(EMA_PERIOD+1)
EPOCHS=3
WARMUP_STEPS = 200
ADAM_MAX_LR = 4e-5

In [20]:
(275/950)

0.2894736842105263

In [39]:
#T_max = 275 is for 1 epoch
lambda1 = lambda x:  x/ WARMUP_STEPS if x < WARMUP_STEPS else 1
optimizer = AdamW(model.parameters(), lr=ADAM_MAX_LR, weight_decay=0.01) #added weight decay
# scheduler = CosineAnnealingWarmRestarts(optimizer, T_0, T_mult=1, eta_min=0, last_epoch=-1, verbose='deprecated')
scheduler_warmup = LambdaLR(optimizer, lr_lambda=lambda1)
scheduler_cosine = CosineAnnealingLR(optimizer, T_max=(208*EPOCHS-WARMUP_STEPS), eta_min=1e-8) #added scheduler


#CHANGED_LAST
# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=200, num_training_steps = 730)

In [40]:
def compose_input_sequence(dp: DatapointComposed, max_len_tokens: int, tokenizer, context_ratio: float = 0.75, 
                           cut_context=True) -> str:
    #here we will probably get a little bit less than a real number of tokens
    approx_max_len_chars = max_len_tokens * 3
    
    if not -1e8 < context_ratio < 1. + 1e8:
        raise ValueError('context_ratio must be between 0 and 1')

    context = dp.context[0]
    completion = dp.completion[0]
    context = '\n'.join([line for line in context.split('\n') if re.match(pattern, line)])
    completion = '\n'.join([line for line in completion.split('\n') if re.match(pattern, line)])

    length_context = int(approx_max_len_chars * context_ratio) + 1
    length_completion = int(approx_max_len_chars * (1 - context_ratio)) + 1

    compl_trim_idx = completion.rfind('\n', 0, length_completion)
    context_trim_idx = context.find('\n', len(context)-length_context, len(context))

    if compl_trim_idx > 0:
        # print("First cut of completion")
        completion_trimmed = completion[:compl_trim_idx]
    else:
        # print("Second cut of completion")
        completion_trimmed = completion[:length_completion]
    if context_trim_idx > 0:
        # print("First cut of context")
        context_trimmed = context[1+context_trim_idx:]
    else:
        # print("Second cut of context")
        context_trimmed = context[-length_context:]
    
    context_len = tokenizer(context_trimmed, return_tensors="pt")['input_ids'].shape[1]
    completion_len = tokenizer(completion_trimmed, return_tensors="pt")['input_ids'].shape[1]
    res_input = tokenizer(context_trimmed + completion_trimmed, return_tensors="pt")
    
    if completion_len>=length_completion:
        print(f"Very strange output: completion_len>=length_completion: {completion_len}>={length_completion}. Skipping")
        print(completion_trimmed)
        return None, None, None
    if context_len>=length_context:
        print(f"Very strange output: context_len>=length_context: {context_len}>={length_context}. Skipping")
        print(completion_trimmed)
        return None, None, None
    # if 2*completion_len>=length_completion:
    #     print(f"Look here: 2*completion_len>=length_completion: 2*{completion_len}>={length_completion}. Not skipping")
    #     print(completion_trimmed)
    # if 2*context_len>=length_context:
    #     print(f"Look here: 2*context_len>=length_context: 2*{context_len}>={length_context}. Not skipping")
    #     print(completion_trimmed)
    
        
    # print(f"context_len: {context_len}, completion_len: {completion_len}")
    # # if completion_len>1000:
    #     # print(completion_trimmed)
    # print(f"length_context:{length_context}, length_completion:{length_completion}, len(context_trimmed): {len(context_trimmed)}, len(completion_trimmed): {len(completion_trimmed)}")
    # print(f"context_trim_idx:{context_trim_idx}, len(context): {len(context)}, compl_trim_idx:{compl_trim_idx}, len(completion): {len(completion)}")
    if len(completion_trimmed)<completion_len:
        print(completion_trimmed)
    diff_tokens = 0
   
    if (context_len+completion_len > max_len_tokens):
        diff_tokens = context_len+completion_len - max_len_tokens
        if cut_context:
            res_input['input_ids'] =res_input['input_ids'][:, diff_tokens:]
            res_input['attention_mask'] =res_input['attention_mask'][:,diff_tokens:]
            context_len = context_len - diff_tokens
        else:
            res_input['input_ids'] =res_input['input_ids'][:, :-diff_tokens]
            res_input['attention_mask'] =res_input['attention_mask'][:,:-diff_tokens]
            completion_len = completion_len - diff_tokens
            
    
    return res_input, context_len, completion_len



First cut of completion
First cut of context
context_len: 1222, completion_len: 
length_context:4501, length_completion:1501, len(context_trimmed): 4474, len(completion_trimmed): 1469
context_trim_idx:995525, len(context): 1000000, compl_trim_idx:1469, len(completion): 20843

In [41]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output


def live_plot(data1, data2, data3, data4, figsize=(10, 20), title1='Train loss on whole sequence (EMA)', title2='Train loss on completion (EMA)',
             title3='Val loss on whole sequence (EMA)', title4='Val loss on completion (EMA)'):
    clear_output(wait=True)
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=figsize)  # Creates 4 subplots vertically aligned

    # Plotting data1 on the first subplot
    ax1.plot(data1, marker='o')
    ax1.set_title(title1)
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Loss')
    ax1.grid(True)

    # Plotting data2 on the second subplot
    ax2.plot(data2, marker='o')
    ax2.set_title(title2)
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Loss')
    ax2.grid(True)

    # Plotting data1 on the first subplot
    ax3.plot(data3, marker='o')
    ax3.set_title(title3)
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Loss')
    ax3.grid(True)

    # Plotting data2 on the second subplot
    ax4.plot(data4, marker='o')
    ax4.set_title(title4)
    ax4.set_xlabel('Iteration')
    ax4.set_ylabel('Loss')
    ax4.grid(True)

    plt.tight_layout()
    plt.show()

In [42]:
class HFDataset(Dataset):
    def __init__(self, hf_dataset):
        self.hf_dataset = hf_dataset
    
    def __len__(self):
        return len(self.hf_dataset)
    
    def __getitem__(self, idx):
        return self.hf_dataset[idx]

dataset = load_dataset("JetBrains-Research/context-py-train", "path_distance_relevant", cache_dir='/mnt/data2/shared-data/lca/hf_cache2/')

Resolving data files:   0%|          | 0/124 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/124 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [43]:
def get_val_dataset(num_points = 128):
    curr_num_points = 0
    validation_repos = set()
    train_repos = list(set(dataset['train']['repo']))
    cnt = Counter(dataset['train']['repo'])
    val_indices_list = []
    while curr_num_points < num_points:  
        curr_repo = random.choice(train_repos)
        num_points_to_add = min(5, num_points-curr_num_points, cnt[curr_repo])
        indices = [i for i, value in enumerate(dataset['train']['repo']) if value == curr_repo]
        val_indices_list.extend(random.sample(indices, num_points_to_add))
        validation_repos.add(curr_repo)
        train_repos.remove(curr_repo)
        curr_num_points = curr_num_points + num_points_to_add
    val_dataset = dataset['train'].select(val_indices_list)
    return val_dataset, validation_repos, train_repos, val_indices_list
    

In [44]:
# val_dataset, validation_repos, train_repos, val_indices_list = get_val_dataset()

In [45]:
# write_val_dataset("v2_all", val_dataset, validation_repos, val_indices_list)

In [46]:
val_dataset, validation_repos, val_indices_list = read_val_dataset("v0")

In [47]:
# Filter function to split based on repositories
def is_validation(entry):
    return entry['repo'] in validation_repos

def is_train(entry):
    return entry['repo'] not in validation_repos

def get_new_ema(curr_loss, prev_ema):
    return (curr_loss - prev_ema)*EMA_ALPHA + prev_ema


In [48]:
train_dataset = dataset['train'].filter(is_train)
torch_train_dataset = HFDataset(train_dataset)
train_loader = DataLoader(torch_train_dataset, batch_size=1, shuffle=True)

torch_val_dataset = HFDataset(val_dataset)
val_loader = DataLoader(torch_val_dataset, batch_size=1, shuffle=True)

In [49]:
print(len(val_dataset))

128


Accelerator():

NotImplementedError: Using RTX 4000 series doesn't support faster communication broadband via P2P or IB. Please set `NCCL_P2P_DISABLE="1"` and `NCCL_IB_DISABLE="1" or use `accelerate launch` which will do this automatically.



In [50]:
NCCL_P2P_DISABLE="1"
NCCL_IB_DISABLE="1"
# os.environ["ACCELERATE_CONFIG"] = "accelerate_config.json"
# accelerator = Accelerator()

In [51]:
# accelerator.state.process_index

In [52]:
# accelerator.state

In [53]:
# !accelerate config 1

In [54]:
os.environ['NCCL_P2P_DISABLE'] = "1"
os.environ['NCCL_IB_DISABLE'] = "1"

devices_to_train_str = ','.join(str(x) for x in DEVICES_TO_TRAIN)
# CHANGED
accelerator = Accelerator(split_batches=False, mixed_precision="no", gradient_accumulation_steps=ACCUM_STEPS_NUM)
model, optimizer, train_loader, val_loader = accelerator.prepare(model, optimizer, train_loader, val_loader)


In [55]:
print(len(train_dataset))

13275


In [56]:
 # hf_path='JetBrains-Research/context-py-train',
#         hf_config=None,
#         cache_dir='/mnt/data2/shared-data/lca/hf_cache2/'

In [57]:
def make_validation_step(model, val_losses_whole_input, val_losses_completion, val_loss_whole_input_ema, val_loss_completion_ema):
    model.eval()  
    with accelerator.autocast(), torch.no_grad():
        total_val_loss = 0
        total_val_tokens = 0
        curr_val_loss_whole_input=[]
        curr_val_loss_completion=[]
        val_loader_iter = iter(val_loader)  # Create an iterator for the validation data

        for idx, val_hf_dp in enumerate(val_loader_iter):

            val_dp = DatapointComposed.from_hf_datapoint(val_hf_dp)
            val_inputs, val_context_len, val_completion_len = compose_input_sequence(val_dp, CONTEXT_MAX_LEN_TOKENS, tokenizer, 0.75)
            if val_inputs is not None:
                val_inputs = val_inputs.to(model.device)
                val_outputs = model.forward(**val_inputs)
                val_logits_size = val_outputs['logits'].size(-1)
                val_loss_completion = criterion(val_outputs['logits'].view(-1, val_outputs['logits'].size(-1))[-val_completion_len:-1, :], val_inputs['input_ids'].view(-1)[-val_completion_len+1:])
                val_detached_logits = val_outputs['logits'].detach()
                
                # for 0 output logit we have corresponding 1st input
                val_loss_whole_input = criterion(val_detached_logits.view(-1, val_detached_logits.size(-1))[:-1, :], val_inputs['input_ids'].view(-1)[1:]) 
                curr_val_loss_whole_input.append(val_loss_whole_input.item())
                curr_val_loss_completion.append(val_loss_completion.item())
                
        #calculate losses
        val_losses_whole_input.append(sum(curr_val_loss_whole_input)/len(curr_val_loss_whole_input))
        val_losses_completion.append(sum(curr_val_loss_completion)/len(curr_val_loss_completion))
        val_loss_whole_input = val_losses_whole_input[-1]
        val_loss_completion = val_losses_completion[-1]
        
        if len(val_losses_whole_input)>EMA_PERIOD:
            val_loss_whole_input_ema.append(get_new_ema(val_losses_whole_input[-1],val_loss_whole_input_ema[-1]))
            val_loss_completion_ema.append(get_new_ema(val_losses_completion[-1],val_loss_completion_ema[-1]))      
        elif len(val_losses_whole_input)==EMA_PERIOD:
            val_loss_whole_input_ema.append(np.mean(val_losses_whole_input))
            val_loss_completion_ema.append(np.mean(val_losses_completion))
        if len(val_losses_whole_input)>=EMA_PERIOD:    
            val_loss_whole_input_ema_val = val_loss_whole_input_ema[-1]
            val_loss_completion_ema_val = val_loss_completion_ema[-1]
            
        else:
            val_loss_whole_input_ema_val=-1
            val_loss_completion_ema_val=-1
    
    model.train()  # Set the model back to train mode
    return val_loss_whole_input, val_loss_completion, val_loss_whole_input_ema_val, val_loss_completion_ema_val

In [58]:
def get_val_loss(model):
    model.eval()  
    with accelerator.autocast(), torch.no_grad():
        total_val_loss = 0
        total_val_tokens = 0
        curr_val_loss_whole_input=[]
        curr_val_loss_completion=[]
        val_loader_iter = iter(val_loader)  # Create an iterator for the validation data

        for idx, val_hf_dp in enumerate(val_loader_iter):

            val_dp = DatapointComposed.from_hf_datapoint(val_hf_dp)
            val_inputs, val_context_len, val_completion_len = compose_input_sequence(val_dp, CONTEXT_MAX_LEN_TOKENS, tokenizer, 0.75)
            if val_inputs is not None:
                val_inputs = val_inputs.to(model.device)
                val_outputs = model.forward(**val_inputs)
                val_logits_size = val_outputs['logits'].size(-1)
                val_loss_completion = criterion(val_outputs['logits'].view(-1, val_outputs['logits'].size(-1))[-val_completion_len:-1, :], val_inputs['input_ids'].view(-1)[-val_completion_len+1:])
                val_detached_logits = val_outputs['logits'].detach()
                
                # for 0 output logit we have corresponding 1st input
                val_loss_whole_input = criterion(val_detached_logits.view(-1, val_detached_logits.size(-1))[:-1, :], val_inputs['input_ids'].view(-1)[1:]) 
                curr_val_loss_whole_input.append(val_loss_whole_input.item())
                curr_val_loss_completion.append(val_loss_completion.item())
                
        return sum(curr_val_loss_whole_input)/len(curr_val_loss_whole_input), sum(curr_val_loss_completion)/len(curr_val_loss_completion)
    

In [59]:
wandb_config={
    "ACCUM_STEPS_NUM": ACCUM_STEPS_NUM,
    "VALIDATION_PERIOD": VALIDATION_PERIOD,
    "CONTEXT_MAX_LEN_TOKENS": CONTEXT_MAX_LEN_TOKENS,
    "EMA_PERIOD": EMA_PERIOD, 
    "VAL_DATASET_SIZE": 128,
    "ADAMW_MAX_LR": 5e-4, 
    "ADAMW_weight_decay":0.01,
    "warmup":200,
    # "Cosine_T_max":50, 
    # "Cosine_eta_min":1e-6,
    # "num_GPU_cards": 2,
    "DEVICES_TO_TRAIN":DEVICES_TO_TRAIN,
    "DeepSpeed_plugin": False
    }

In [60]:
print(wandb_config)

{'ACCUM_STEPS_NUM': 64, 'VALIDATION_PERIOD': 10, 'CONTEXT_MAX_LEN_TOKENS': 2000, 'EMA_PERIOD': 40, 'VAL_DATASET_SIZE': 128, 'ADAMW_MAX_LR': 0.0005, 'ADAMW_weight_decay': 0.01, 'warmup': 200, 'DEVICES_TO_TRAIN': [3], 'DeepSpeed_plugin': False}


In [61]:
wandb.init(
    # set the wandb project where this run will be logged
    project="finetuning_with_lora",
    # track hyperparameters and run metadata
    config=wandb_config
)


In [62]:
# https://github.com/huggingface/accelerate/blob/main/examples/by_feature/deepspeed_with_config_support.py

In [63]:
optimizer._is_accelerate_prepared

True

In [64]:
# dir(optimizer)

In [65]:
ACCUM_STEPS_NUM*50

3200

https://pytorch.org/torchtune/stable/generated/torchtune.modules.get_cosine_schedule_with_warmup.html

https://huggingface.co/docs/transformers/en/main_classes/optimizer_schedules#transformers.get_cosine_schedule_with_warmup

In [66]:
def train(model):
    torch.cuda.empty_cache()
    total_whole_input_loss = 0
    total_completion_loss = 0
    total_tokens = 0
    total_completion_tokens = 0
    train_losses_whole_input = []
    curr_train_loss_whole_input = []
    train_losses_completion = []
    curr_train_loss_completion = []
    
    val_losses_whole_input = []
    val_losses_completion = []
    
    
    train_loss_whole_input_ema = []
    train_loss_completion_ema = []
    val_loss_whole_input_ema = []
    val_loss_completion_ema = []
    wandb_step=1
    
    val_loss_whole_input, val_loss_completion = get_val_loss(model)
    # CHANGED
    checkpoint_num = 0
    save_checkpoint(checkpoint_num, model, val_loss_whole_input, val_loss_completion, val_dataset, suffix=SUFFIX)
    checkpoint_num=checkpoint_num+1
    scheduler_type=0
    scheduler_step=1

    for epoch in range(EPOCHS):
        train_loader_iter = iter(train_loader)
    
        for idx, hf_dp in enumerate(train_loader_iter):
            # torch.cuda.empty_cache()
            # if idx > 20:
            #     break
        
            dp = DatapointComposed.from_hf_datapoint(hf_dp)
            inputs, context_len, completion_len = compose_input_sequence(dp, CONTEXT_MAX_LEN_TOKENS, tokenizer, 0.75)
            if inputs is not None:
                assert abs(completion_len+context_len-inputs['input_ids'].shape[1])<2
                # full_len = inputs['input_ids'].shape[1]
                inputs = inputs.to(model.device)
                outputs = model.forward(**inputs)
                logits_size = outputs['logits'].size(-1)
                loss_completion = criterion(outputs['logits'].view(-1, outputs['logits'].size(-1))[-completion_len:-1, :], inputs['input_ids'].view(-1)[-completion_len+1:])
                # CHANGED
                accelerator.backward(loss_completion)
                # loss_completion.backward()
                detached_logits = outputs['logits'].detach()
                
                # for 0 output logit we have corresponding 1st input
                loss_whole_input = criterion(detached_logits.view(-1, detached_logits.size(-1))[:-1, :], inputs['input_ids'].view(-1)[1:]) 
                
                # calculate losses and backprop
                if (idx+1) % ACCUM_STEPS_NUM ==0:
                    train_losses_whole_input.append(sum(curr_train_loss_whole_input)/len(curr_train_loss_whole_input))
                    train_losses_completion.append(sum(curr_train_loss_completion)/len(curr_train_loss_completion))

                    train_loss_whole_input_ema_val = 0
                    train_loss_completion_ema_val = 0
                    if len(train_losses_whole_input)>EMA_PERIOD:
                        train_loss_whole_input_ema_val = get_new_ema(train_losses_whole_input[-1],train_loss_whole_input_ema[-1])
                        train_loss_completion_ema_val = get_new_ema(train_losses_completion[-1],train_loss_completion_ema[-1])
                    elif len(train_losses_whole_input)==EMA_PERIOD:
                        train_loss_whole_input_ema_val = np.mean(train_losses_whole_input)
                        train_loss_completion_ema_val = np.mean(train_losses_completion)
                    else:
                        train_loss_whole_input_ema_val = -1
                        train_loss_whole_completion_ema_val = -1

                    if len(train_losses_whole_input)>=EMA_PERIOD:
                        train_loss_whole_input_ema.append(train_loss_whole_input_ema_val)
                        train_loss_completion_ema.append(train_loss_completion_ema_val)
        
                    curr_train_loss_whole_input = []
                    curr_train_loss_completion=[]
                    for param in model.parameters():
                        if param.requires_grad:
                            if param.grad is not None:
                                param.grad /= ACCUM_STEPS_NUM
                    
                    optimizer.step()
                    optimizer.zero_grad()      
                    
                    if scheduler_step < WARMUP_STEPS:
                        scheduler_warmup.step()
                        curr_lr = scheduler_warmup.get_lr()[0]
                        scheduler_type=0
                    else:
                        scheduler_cosine.step()
                        curr_lr = scheduler_cosine.get_lr()[0]
                        scheduler_type=1
                    if train_loss_whole_input_ema_val>0:    
                        wandb.log({"scheduler_lrr": curr_lr, "scheduler_type": scheduler_type, "scheduler_step": scheduler_step, 
                                   "epoch":epoch,
                                  "train_loss_whole_input": train_losses_whole_input[-1], 
                                  "train_loss_completion": train_losses_completion[-1],
                                  "train_loss_whole_input_ema": train_loss_whole_input_ema_val, 
                                  "train_loss_completion_ema": train_loss_completion_ema_val}, step = wandb_step)
                    else:
                       wandb.log({"scheduler_lrr": curr_lr, "scheduler_type": scheduler_type, "scheduler_step": scheduler_step, 
                                   "epoch":epoch,
                                  "train_loss_whole_input": train_losses_whole_input[-1], 
                                  "train_loss_completion": train_losses_completion[-1]}, step = wandb_step) 
                    scheduler_step = scheduler_step+1
        
                #we could set some other parameter here
                # if ((idx+1) % ACCUM_STEPS_NUM*VALIDATION_PERIOD) ==0:
                if ((idx+1) % ACCUM_STEPS_NUM) ==0:
                    val_loss_whole_input_val, val_loss_completion_val, val_loss_whole_input_ema_val, val_loss_completion_ema_val = make_validation_step(model, val_losses_whole_input, val_losses_completion, val_loss_whole_input_ema, val_loss_completion_ema)
                    if val_loss_whole_input_ema_val>0:
                        wandb.log({"val_loss_whole_input":val_loss_whole_input_val, "val_loss_completion": val_loss_completion_val, 
                                  "val_loss_whole_input_ema": val_loss_whole_input_ema_val, 
                                   "val_loss_completion_ema": val_loss_completion_ema_val}, step = wandb_step)
                    else:
                        wandb.log({"val_loss_whole_input":val_loss_whole_input_val, "val_loss_completion": val_loss_completion_val}, step = wandb_step)

                if ((idx+1) % ACCUM_STEPS_NUM) ==0:
                    wandb_step = wandb_step+1
                    
        
                if ((idx+1) % (ACCUM_STEPS_NUM*50)) ==0:
                    # CHANGED
                    save_checkpoint(checkpoint_num, model, val_losses_whole_input[-1], val_losses_completion[-1], val_dataset, suffix=SUFFIX)
                    checkpoint_num=checkpoint_num+1
                    
                    
            
                # if (idx+1) % (ACCUM_STEPS_NUM*5) ==0:
                #     if (len(train_loss_whole_input_ema)>2) and (len(val_loss_whole_input_ema)>2):
                #         live_plot(train_loss_whole_input_ema, train_loss_completion_ema, val_loss_whole_input_ema, val_loss_completion_ema)
                #         print(f"Plotted: {(idx+1)} steps")
        
                    
                
                curr_train_loss_whole_input.append(loss_whole_input.item())
                curr_train_loss_completion.append(loss_completion.item())
                total_whole_input_loss += loss_whole_input.item() * (inputs['input_ids'].size(0)-1)  # Accumulate scaled loss
                total_completion_loss += loss_completion.item() * (completion_len -1)  # Accumulate scaled loss
                total_tokens += len(inputs['input_ids']) - 1  # Count tokens processed
                total_completion_tokens += completion_len - 1
    
            
    
            
            
    avg_loss = total_whole_input_loss / total_tokens
    perplexity = math.exp(avg_loss)  # Calculate perplexity as exp of the average loss
    
    avg_completion_loss = total_completion_loss / total_completion_tokens
    perplexity = math.exp(avg_completion_loss)
    



    

In [67]:
# wandb.log({"scheduler_lrr":scheduler.get_lr()[0]})

In [68]:
print(SUFFIX)

lr4e-5_old_scheduler_with_warmup_3_epochs_relevant


In [ ]:
train(model)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
print(checkpoint_num)

In [ ]:
wandb.finish()

First cut of completion
First cut of context
context_len: 1222, completion_len: 2548
length_context:4501, length_completion:1501, len(context_trimmed): 4474, len(completion_trimmed): 1469
context_trim_idx:995525, len(context): 1000000, compl_trim_idx:1469, len(completion): 20843